In [1]:
filename = 'FACTURACIÓN PENDIENTE NIC18 FEBRERO 20220419.xlsx'
year = 2022

# Libraries

In [2]:
import pandas as pd
import pyodbc
import os
from datetime import datetime

pd.io.formats.excel.ExcelFormatter.header_style = None
pd.options.display.float_format = '{:,.2f}'.format

# Parameters

In [3]:
path = r'\\NASPRO.infovisiontv.com\DGI\DGI Banco Central\00 CIERRES SEMANALES\{0}\NIC18'
path = path.format(year)

# Import Data

'Grupo de artículos'

In [4]:
%%time
df = pd.read_excel(path + '\\' + filename, sheet_name='YOSAF', dtype={'Grupo de artículos':'str', 'Tp.cambio p.contab.':'str', 'Lote':'str'})

Wall time: 3.81 s


df[['Grupo de artículos', 'Tp.cambio p.contab.', 'Lote']].dtypes

In [5]:
df.shape

(1866, 47)

# Data Cleaning

df = df[:df.shape[0]-6]

In [6]:
df.isnull().sum()[df.isnull().sum() > 0]

Cd.Barra                      136
RFC                           322
Nº cuenta anterior             61
Motivo pedido                1866
Descripción Motivo Pedido    1866
dtype: int64

In [7]:
new_cols = [
     'Factura',
     'Posicion',
     'DocumentoVentas',
     'PosicionDV',
     'NumeroPedCliente',
     'Material',
     'CodigoBarras',
     'Denominacion',
     'Solicitante',
     'Nombre',
     'RFC',
     'CNAC',
     'GrupoCuentas',
     'GrupoTesoreria',
     'DestinatarioMcia',
     'NombreDestinatario',
     'CantidadFacturada',
     'UnidadMedida',
     'CtdFactUMB',
     'UnidadMedidaBase',
     'PrecioUnitario',
     'ImpuestoBase',
     'ImpuestoDescuento',
     'ValorNeto',
     'ImporteImpuesto',
     'ImpuestoIEPS',
     'ImpuestoTotal',
     'CentroBeneficio',
     'DescripcionCebe',
     'TipoUnidades',
     'FechaFactura',
     'GrupoArticulos',
     'DenominacionArticulos',
     'ClaseFactura',
     'DenominacionTipoDocto',
     'CambioContable',
     'Sociedad',
     'Nombreempresa',
     'OrganizacionVentas',
     'Centro',
     'Almacen',
     'Lote',
     'FechaCaducidad',
     'MotivoPedido',
     'DescMotivoPedido',
     'ValorNetoAbs',
     'Moneda'
]

In [8]:
df.columns = new_cols

In [9]:
cols_nulls = [
    'RFC', 
    'CNAC', 
    'Almacen', 
    'Lote', 
    'MotivoPedido', 
    'DescMotivoPedido'
]

for col in cols_nulls: 
    df[col].fillna('     ', inplace=True)

In [10]:
df.isnull().sum()[df.isnull().sum() > 0]

CodigoBarras    136
dtype: int64

In [11]:
cols_int = [
    'Factura',
    'Posicion',
    'DocumentoVentas',
    'PosicionDV',
    'Material',
    'Solicitante',
    'CtdFactUMB',
    'ImpuestoDescuento',
    'ImpuestoIEPS',
]

In [12]:
for col in cols_int:
    df[col] = df[col].map(int)

In [13]:
df[['FechaCaducidad', 'FechaCaducidad']].dtypes

FechaCaducidad    datetime64[ns]
FechaCaducidad    datetime64[ns]
dtype: object

# Analysis

In [14]:
print('FechaFactura desde:', df['FechaFactura'].min())
print('FechaFactura hasta:', df['FechaFactura'].max())

FechaFactura desde: 2022-02-28 00:00:00
FechaFactura hasta: 2022-02-28 00:00:00


In [15]:
print('FechaCaducidad desde:', df['FechaCaducidad'].min())
print('FechaCaducidad hasta:', df['FechaCaducidad'].max())

FechaCaducidad desde: 2022-09-30 00:00:00
FechaCaducidad hasta: 2027-02-28 00:00:00


# Export Data

In [16]:
df.to_excel(path + '\\Cargar ' + filename.split('.')[0] + '.xlsx',
            index=False,
            sheet_name='carga')

In [17]:
df.shape

(1866, 47)

# Validations

In [18]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Trusted_Connection=yes;')

In [19]:
qtime = '''SELECT DISTINCT 
                TmpFecha,
                TmpSemanaAnioGenomma AS SemanaGL,
                TmpAnioSemanaGenomma AS AnioGL
            FROM Gnm_DWH.dbo.Dim_Tiempo'''

In [20]:
dtime = pd.read_sql(qtime, conn)

In [21]:
df = df.merge(dtime, left_on='FechaFactura', right_on='TmpFecha', how='left')

In [22]:
df.pivot_table(
    index='SemanaGL', 
    values=['ValorNeto'],
    aggfunc='sum'
)

,ValorNeto
SemanaGL,
10,"44,240,996.99"


In [23]:
df.pivot_table(
    index='SemanaGL', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
SemanaGL,
10,1866
All,1866


In [24]:
df.pivot_table(
    index='FechaFactura', 
    values='Factura',
    aggfunc='count',
    margins=True
)

,Factura
FechaFactura,
2022-02-28 00:00:00,1866
All,1866


In [25]:
df.pivot_table(
    index='MotivoPedido', 
    values='ValorNeto', 
    aggfunc='sum'
)

,ValorNeto
MotivoPedido,
,"44,240,996.99"
